In [1]:
import pandas as pd
import gzip



In [2]:
df = pd.read_csv(gzip.open('data/manipulation/combined_and_tidied_geolocated_data.csv.gz', 'rb'))
df.tail()

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7,10,13,16,48,52) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index,id,relid,year,active_year,code_status,type_of_violence,conflict_dset_id,conflict_new_id,conflict_name,...,deaths_civilians,deaths_unknown,best,high,low,gwnoa,gwnob,country_name,method,flag
279266,1675,413661,TAZ-2021-1-16474-0,2021,0,Clear,1,15183,15183,Tanzania: Islamic State,...,0,1,5,5,0,510.0,NaN,Tanzania,Nearest,NaN
279267,1676,386795,YEM-2021-3-1182-17,2021,1,Clear,3,123,715,Government of Yemen (North Yemen) - Civilians,...,1,0,1,1,0,678.0,NaN,Saudi Arabia,Nearest,NaN
279268,1677,389676,YEM-2021-1-14595-113,2021,1,Clear,1,230,230,Yemen (North Yemen): Government,...,1,0,1,1,1,678.0,NaN,Yemen,Nearest,NaN
279269,1678,402883,YEM-2021-1-14595-202,2021,1,Clear,1,230,230,Yemen (North Yemen): Government,...,1,0,1,1,1,678.0,NaN,Yemen,Nearest,NaN
279270,1679,419727,YEM-2021-1-14595-523,2021,1,Clear,1,230,230,Yemen (North Yemen): Government,...,9,0,9,12,8,678.0,NaN,Yemen,Nearest,NaN


Aggregate by country and year